## Instalando Bibliotecas

In [ ]:
%pip install -qU google-generativeai langchain-google-genai
%pip install -qU pydantic

## Configurando Gemini

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from pydantic import BaseModel, Field
from typing import Literal, List, Optional

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
print(llm.invoke("Olá tudo bem ?"))

content='Olá! Tudo bem por aqui, obrigado por perguntar! 😊 Como posso te ajudar hoje?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-3addd157-b6b8-418b-a50b-5bbb95f7b382-0' usage_metadata={'input_tokens': 4, 'output_tokens': 20, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}}


## Saida Estruturada Simples: Extraindo Dados de um Email

In [ ]:
class reuniao(BaseModel):
    data: str
    hora: str
    local: str = Field(description="Local da reunião")
    assunto: Optional[str] = Field(default="Email", description="Assunto do email")

email_llm = llm.with_structured_output(reuniao)

email_reuniao = """
Olá pessoal,
Nossa reunião do projeto X foi marcada para 15/04/2025 às 15h, link do meet: meet.com.br/reuniao.
Estão confirmados: João, Maria e Ana.
Vamos discutir os próximos passos do lançamento.
Até lá!
"""
email = email_llm.invoke(email_reuniao)
print(email)

In [ ]:
print(email.hora)

15h


## Multi Agentes Simples com Saida Estruturada: Monitorando Emails

In [ ]:
class tipo_email(BaseModel):
    tipo: Literal["Reuniao", "Agendamento de Reuniao", "Pedido de Vendas", "Outro", "Spam"]

classificador = llm.with_structured_output(tipo_email)

email_reuniao = """
Olá pessoal,
Nossa reunião do projeto X foi marcada para 15/04/2025 às 15h, link do meet: meet.com.br/reuniao.
Estão confirmados: João, Maria e Ana.
Vamos discutir os próximos passos do lançamento.
Até lá!
"""
classificacao = classificador.invoke(email_reuniao)
print(classificacao)

### Criando Modelos e Funções

In [ ]:
# MODELOS ESTRUTURADOS
class Reuniao(BaseModel):
    data: str
    hora: str
    local: str
    assunto: str

class Produto(BaseModel):
    nome: str
    quantidade: int
    valor: float

class Pedido(BaseModel):
    produtos: List[Produto]
    valor_total: float

# FUNÇÕES
def simular_reuniao(email: str) -> str:
    extrair_reuniao = llm.with_structured_output(Reuniao)
    reuniao = extrair_reuniao.invoke(email)
    print("Resposta gerada:", reuniao)
    return (
        f"✅ Evento criado com sucesso:\n"
        f"Assunto: {reuniao.assunto}\n"
        f"📅 {reuniao.data} às ⏰ {reuniao.hora}\n"
        f"📍 Local: {reuniao.local}"
    )

def simular_agendamento(email: str) -> str:
    horarios_ocupados = """
    Sábado: 10h, 11h, 12h
    Segunda-feira: 10h, 11h, 12h
    Terça-feira: 10h, 11h, 12h
    Quarta-feira: 10h, 11h, 12h
    """

    prompt = (
        "Você é um assistente de agendamento. "
        "Verifique se o horário solicitado no email abaixo está disponível. "
        "Se estiver, envie um email confirmando o agendamento. "
        "Se não estiver, envie um email informando indisponibilidade e sugira horários livres.\n\n"
        "Retorne apenas o conteudo do email sem dados extras"
        f"Email recebido:\n{email}\n"
        f"Horários Ocupados:\n{horarios_ocupados}"
    )

    resposta = llm.invoke(prompt).content
    print("Resposta gerada:", resposta)

    return f"📩 Email enviado:\n{resposta}"

def simular_pedido_vendas(email: str) -> str:
    extrair_pedido = llm.with_structured_output(Pedido)
    pedido = extrair_pedido.invoke(email)
    print("Resposta gerada:", pedido)
    lista_produtos = "\n".join(
        [f"- {p.nome} (x{p.quantidade}) - R$ {p.valor:.2f}" for p in pedido.produtos]
    )
    return f"🛒 Pedido Recebido:\n{lista_produtos}\n💰 Valor Total: R$ {pedido.valor_total:.2f}"

def invoke(email: str) -> str:
    classificacao = classificador.invoke(email)
    classificacao = classificacao.tipo
    print("Classificação:", classificacao)
    if classificacao == "Reuniao":
        return simular_reuniao(email)
    elif classificacao == "Agendamento de Reuniao":
        return simular_agendamento(email)
    elif classificacao == "Pedido de Vendas":
        return simular_pedido_vendas(email)
    elif classificacao == "Outro":
        return "🤖 Email encaminhado para análise humana."
    else:
        return "🚫 Spam detectado e excluído."


### Templates de email

In [ ]:
email_reuniao = """
Olá equipe,

Gostaria de confirmar nossa reunião sobre o projeto X para amanhã às 15h, na sala 203.

Assunto: Alinhamento do Projeto X

Atenciosamente,
João
"""

email_agendamento = """
Prezados,

Gostaria de agendar uma reunião na terça-feira às 11h para discutirmos o planejamento trimestral.

Aguardo confirmação.

Abraços,
Fernanda
"""

email_pedido = """
Olá, gostaria de fazer um pedido com os seguintes produtos:

- Teclado Mecânico, 2 unidades, R$ 250,00 cada
- Mouse Sem Fio, 1 unidade, R$ 120,00

Favor confirmar o valor total e o prazo de entrega.

Grato,
Carlos
"""

email_outro = """
Boa tarde,

Gostaria de entender mais sobre os serviços que vocês oferecem em consultoria de TI. Poderiam me enviar uma apresentação?

Obrigado.
"""

email_spam = "🤑💰 Ganhe dinheiro rápido! Clique aqui para saber mais!!"


### Chamando llm

In [ ]:
resposta = invoke(email_pedido)
print(resposta)
